In [1]:
ls

chv_rus_1M_CorpusWiki_data/   corpus_wiki.ipynb      subword-nmt/
chv_rus_1M_CorpusWiki_model/  corpus_wiki_train.csv  wiki_train_all.csv
corpus_test.csv               data/
corpus_train_all.csv          sockeye/


In [2]:
import sockeye
import tensorboard

In [3]:
import torch
print(torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_name(i))

/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1
NVIDIA A100-SXM4-80GB


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import load_dataset

In [5]:
src = 'chv'
tgt = 'rus'
name = '1M_CorpusWiki'

In [41]:
train_path = 'corpus_wiki_train.csv'
test_path = 'corpus_test.csv'

df_test = pd.read_csv(test_path)
df_train = load_dataset('csv', data_files=train_path).shuffle(seed=42)

split_dataset = df_train['train'].train_test_split(test_size=0.1)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 152.54it/s]
Generating train split: 0 examples [00:00, ? examples/s]/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/kvzhirnov/.cache/huggingface/datasets/csv/default-858b36ba709bbb36/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 11.14it/s]


In [42]:
!mkdir 'data'
!mkdir 'data/train'
!mkdir 'data/dev'
!mkdir 'data/test'

In [43]:
def writelines(list, filename):
  with open(filename, 'w', encoding='utf-8') as file:
    for item in list:
      file.write('%s\n' % item)

chv_train = [l['chv'] for l in split_dataset['train']]
rus_train = [l['rus'] for l in split_dataset['train']]

chv_dev = [l['chv'] for l in split_dataset['test']]
rus_dev = [l['rus'] for l in split_dataset['test']]

chv_test = df_test['chv'].tolist()
rus_test = df_test['rus'].tolist()

writelines(chv_train, 'data/train/train.chv.txt')
writelines(rus_train, 'data/train/train.rus.txt')

writelines(chv_dev, 'data/dev/dev.chv.txt')
writelines(rus_dev, 'data/dev/dev.rus.txt')

writelines(chv_test, 'data/test/test.chv.txt')
writelines(rus_test, 'data/test/test.rus.txt')

In [44]:
file_name = 'data/train/train.chv.txt'
number_of_rows = 5  # The number of rows you want to read

with open(file_name, 'r') as file:
    for i in range(number_of_rows):
        line = file.readline()
        print(line.strip())

— Пӗр лаша та юлмасть вӗт!
Хӑйсем пирки: эпир восстание хирӗҫ, теҫҫӗ.
Анне пӗрмаях «Юрамасть!
Хыҫҫӑнах пысӑк карапӑн тӗрлӗ кӗтесӗнче унтер-офицерсен командисем илтӗнме пуҫлаҫҫӗ.
Эпӗ халех, ҫак самантрах тӑратӑп, — айӑпа кӗнӗ пек пӑшӑлтатса илтӗм эпӗ.


In [45]:
file_name = 'data/train/train.rus.txt'
number_of_rows = 5  # The number of rows you want to read

with open(file_name, 'r') as file:
    for i in range(number_of_rows):
        line = file.readline()
        print(line.strip())

— Ни черта не останется!
А о себе: мы против восстания.
От постоянного маминого: «Нельзя!
И сейчас же в разных уголках огромного корабля заливались дудки унтер-офицеров и раздавалась команда.
Я сейчас, я сию минуточку, — виновато прошептал я.


In [6]:
!python subword-nmt/subword_nmt/learn_joint_bpe_and_vocab.py --input data/train/train.{src}.txt data/train/train.{tgt}.txt  -s 10000 -o data/bpe.codes  --write-vocabulary data/bpe.vocab.{src} data/bpe.vocab.{tgt}

100%|#####################################| 10000/10000 [03:40<00:00, 45.31it/s]


In [7]:
!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{src} --vocabulary-threshold 50 < data/train/train.{src}.txt > data/train/train.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{tgt} --vocabulary-threshold 50 < data/train/train.{tgt}.txt > data/train/train.{tgt}.bpe

!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{src} --vocabulary-threshold 50 < data/dev/dev.{src}.txt > data/dev/dev.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{tgt} --vocabulary-threshold 50 < data/dev/dev.{tgt}.txt > data/dev/dev.{tgt}.bpe

!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{src} --vocabulary-threshold 50 < data/test/test.{src}.txt > data/test/test.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{tgt} --vocabulary-threshold 50 < data/test/test.{tgt}.txt > data/test/test.{tgt}.bpe

In [20]:
!python -m sockeye.prepare_data -s data/train/train.{src}.bpe -t data/train/train.{tgt}.bpe --shared-vocab -o {src}_{tgt}_{name}_data

[INFO:sockeye.utils] Sockeye: 3.1.34, commit 4c30942ddb523533bccb4d2cbb3e894e45b1db93, path /home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1 (/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/prepare_data.py -s data/train/train.chv.bpe -t data/train/train.rus.bpe --shared-vocab -o chv_rus_1M_CorpusWiki_data
[INFO:sockeye.utils] Arguments: Namespace(config=None, source='data/train/train.chv.bpe', source_factors=[], source_factors_use_source_vocab=[], target_factors=[], target_factors_use_target_vocab=[], target='data/train/train.rus.bpe', end_of_prepending_tag=None, source_vocab=None, target_vocab=None, source_factor_vocabs=[], target_factor_vocabs=[], shared_vocab=True, num_words=(0, 0), word_min_count=(1, 1), pad_vocab_to_multiple_of=8, no_bucketing=False, bucket_width=8, bucket_sc

In [21]:
!python -m sockeye.train -d {src}_{tgt}_{name}_data -vs data/dev/dev.{src}.bpe -vt data/dev/dev.{tgt}.bpe --encoder transformer --decoder transformer --transformer-model-size 512 --transformer-feed-forward-num-hidden 2048 --transformer-dropout-prepost 0.1 --num-embed 512 --max-seq-len 100 --decode-and-evaluate 500 -o {src}_{tgt}_{name}_model --num-layers 6 --keep-last-params 3 --batch-size 2048 --optimized-metric bleu --max-num-checkpoint-not-improved 5 --amp

[INFO:sockeye.utils] Sockeye: 3.1.34, commit 4c30942ddb523533bccb4d2cbb3e894e45b1db93, path /home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1 (/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/train.py -d chv_rus_1M_CorpusWiki_data -vs data/dev/dev.chv.bpe -vt data/dev/dev.rus.bpe --encoder transformer --decoder transformer --transformer-model-size 512 --transformer-feed-forward-num-hidden 2048 --transformer-dropout-prepost 0.1 --num-embed 512 --max-seq-len 100 --decode-and-evaluate 500 -o chv_rus_1M_CorpusWiki_model --num-layers 6 --keep-last-params 3 --batch-size 2048 --optimized-metric bleu --max-num-checkpoint-not-improved 5 --amp
[INFO:sockeye.utils] Arguments: Namespace(config=None, source=None, source_factors=[], source_factors_use_source_vocab=[], target_factors=[], target_fa

In [6]:
!python -m sockeye.translate -m {src}_{tgt}_{name}_model -i data/test/test.{src}.bpe -o data/test/test.{tgt}.pred.bpe --batch-size 128 --beam-size 7

[INFO:sockeye.utils] Sockeye: 3.1.34, commit 4c30942ddb523533bccb4d2cbb3e894e45b1db93, path /home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1 (/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/translate.py -m chv_rus_1M_CorpusWiki_model -i data/test/test.chv.bpe -o data/test/test.rus.pred.bpe --batch-size 128 --beam-size 7
[INFO:sockeye.utils] Arguments: Namespace(config=None, input='data/test/test.chv.bpe', input_factors=None, json_input=False, output='data/test/test.rus.pred.bpe', models=['chv_rus_1M_CorpusWiki_model'], checkpoints=None, nbest_size=1, beam_size=7, greedy=False, beam_search_stop='all', batch_size=128, chunk_size=None, sample=None, seed=None, ensemble_mode='linear', bucket_width=10, max_input_length=None, max_output_length_num_stds=2, max_output_length=None, restrict

In [7]:
!sed -r 's/(@@ )|(@@ ?$)//g' data/test/test.{tgt}.pred.bpe > data/test/test.{tgt}.pred.txt

In [9]:
!sacrebleu data/test/test.rus.txt -tok none -i data/test/test.{tgt}.pred.txt

{
 "name": "BLEU",
 "score": 15.5,
 "signature": "nrefs:1|case:mixed|eff:no|tok:none|smooth:exp|version:2.3.1",
 "verbose_score": "43.6/21.1/11.6/6.7 (BP = 0.948 ratio = 0.949 hyp_len = 1032856 ref_len = 1088181)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "none",
 "smooth": "exp",
 "version": "2.3.1"
}


In [10]:
file_name = 'data/test/test.rus.txt'
number_of_rows = 5  # The number of rows you want to read

with open(file_name, 'r') as file:
    for i in range(number_of_rows):
        line = file.readline()
        print(line.strip())

И если бы не усталость, если бы не потертости, нарывы (а у меня грыжа), я, пожалуй, снова пожелал бы пережить дни нашего немыслимого похода.
— Вот как официально ты представляешься!
Мы падали на дорогу и следили за ними.
Сквозь балки лесов поблёскивала металлическая, с частой клёпкой, поверхность сферического тела.
Вечером в нас стреляли.


In [11]:
file_name = 'data/test/test.rus.pred.txt'
number_of_rows = 5  # The number of rows you want to read

with open(file_name, 'r') as file:
    for i in range(number_of_rows):
        line = file.readline()
        print(line.strip())

Если бы не усталость, если бы не вырытые ремнями (притом у меня есть грыжа), я, может быть, еще больше мечтал бы прожить дни нашего тяжелого похода.
— Вот какого официального ты знакомишь с собой!
Мы растянулись на дороге и смотрели на ракеты.
Сквозь лестничные перекладины поблескивает многоклепчатый металлический слой сферы.
Вечером в эти дни вы нас стреляете!
